In [9]:
from huggingface_hub import login
import os
from dotenv import load_dotenv

load_dotenv()
hf_token = os.getenv("HUGGING_FACE_TOKEN")
login(token=hf_token)

In [10]:
from perplexity import Perplexity
from dotenv import load_dotenv

load_dotenv() 

client = Perplexity() # PERPLEXITY_API_KEY is read from environment variable

search = client.search.create(
    query=[
      "What is Comet Browser?",
      "Perplexity AI",
      "Perplexity Changelog"
    ]
)

for result in search.results:
    print(f"{result.title}: {result.url}")

Introducing Comet: Browse at the speed of thought: https://www.perplexity.ai/hub/blog/introducing-comet
Browse at the speed of thought: https://www.perplexity.ai/comet/
Comet Browser Review 2025: Features, Pricing, Pros & Cons: https://efficient.app/apps/comet
Perplexity's AI-Native Browser Comet is HERE: https://www.youtube.com/watch?v=ri_bFrDp44M
What Is Comet and Why Everyone's Talking About It: https://apidog.com/blog/what-is-comet/
10 Comet Browser Features That Make Chrome & Edge Look ANCIENT: https://www.youtube.com/watch?v=BfC_Lk3NVSs
The Revolutionary AI-Powered Browser That Actually Does Your Work: https://dev.to/s_chathuranga_j/comet-browser-the-revolutionary-ai-powered-browser-that-actually-does-your-work-n6d
Perplexity Comet AI Browser 101: Complete Guide with 100 ...: https://sidsaladi.substack.com/p/perplexity-comet-ai-browser-101-complete
10 Incredible Things You Can Do with Comet — The AI Browser ...: https://blog.truelancer.com/10-things-you-can-do-with-comet-the-ai-b

In [ ]:
import json
from pprint import pprint

goal = "Analyze APPL stock for a potential investment."

prompt = f"""
    As an autonomous investment research agent, create a detailed plan to achieve this goal:
    Goal: {goal}
    
    Available tools: web search, financial data API, news aggregator, sentiment analysis tool

    Create a plan with:
    1. Logical research steps (3-7 steps)
    2. Estimated time in minutes
    3. Required tools for each step
    4. Success criteria to evaluate completion
    
    Answer only in JSON format:
    {{
        "goal": "...",
        "steps": ["step1", "step2", ...],
        "estimated_time": 30,
        "required_tools": ["tool1", "tool2", ...],
        "success_criteria": ["criteria1", "criteria2", ...]
    }}
"""


response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="sonar",
)


### Presenting the data generated

In [15]:
import re


content = response.choices[0].message.content
print("Raw Response:\n", content)

try:
    # quick check if content is already valid JSON
    json.loads(content)
except json.JSONDecodeError:
    # try to extract the first JSON object/array from the text
    m = re.search(r'(\{(?:.|\n)*\}|\[(?:.|\n)*\])', content)
    if m:
        json_text = m.group(1)
        response.choices[0].message.content = json_text  # overwrite so the following json.loads will succeed
        print("Extracted JSON for parsing.")
    else:
        print("No JSON object/array found in the response; parsing will fail.")
        response.choices[0].message.content = "null"
plan_data = json.loads(response.choices[0].message.content)

pprint(plan_data)

Raw Response:
 {
    "goal": "Analyze Apple Inc. (AAPL) stock for a potential investment",
    "steps": [
        "Gather and analyze historical and recent financial performance data (revenue, profitability, cash flow, debt levels)",
        "Conduct a technical analysis of stock price trends and volatility using historical price and trading volume data",
        "Evaluate market sentiment and news regarding Apple’s business, products, and industry risks",
        "Compare Apple’s valuation ratios (P/E, P/B, etc.) with industry peers and perform intrinsic valuation using DCF and relative valuation methods",
        "Assess key investment risks (market, credit, liquidity, operational)",
        "Review Wall Street analyst consensus and price targets",
        "Synthesize findings and make an informed investment recommendation"
    ],
    "estimated_time": 120,
    "required_tools": [
        "Financial data API (Yahoo Finance, Bloomberg, etc.)",
        "Web search for latest reports, e

### Sentiment analysis using perplexity API

#### Get information from FinTools

In [16]:
from src.ingestion import FinancialDataIngestion, NewsDataIngestion

ticker = "PLTR"  # Example stock ticker
stock_info = FinancialDataIngestion().execute(ticker)
pprint(stock_info)

news_articles = NewsDataIngestion().execute(ticker, 5)
for article in news_articles:
    pprint(article)


{'data': {'52_week_high': 190.0,
          '52_week_low': 40.36,
          'avg_volume_30d': 55256481.81818182,
          'beta': 2.595,
          'company_summary': 'Palantir Technologies Inc. builds and deploys '
                             'software platforms for the intelligence '
                             'community to assist in counterterrorism '
                             'investigations and operations in the United '
                             'States, the United Kingdom, and internationally. '
                             'It provides Palantir Gotham, a software '
                             'platform, which enables users to identify '
                             'patterns hidden deep within datasets, ranging '
                             'from signals intelligence sources to reports '
                             'from confidential informants, as well as '
                             'facilitates the hand-off between analysts and '
                             'op

#### Generate sentiment analysis and confidence score

In [18]:

goal = f"Analyze {ticker} stock for a potential investment."

prompt = f"""
    As an autonomous investment research agent, create short sentiment analysis
      about the stock based on the context and recent news.
      Describe the sentiment as Positive, Negative, or Neutral.
      Give a summary of your analysis, and a confidence score between 0 and 1.
    Goal: {goal}

    Context: {stock_info}

    Recent News: {news_articles}

    Answer only in JSON format:
    {{
        "stock": "...",
        "sentiment": "Positive",
        "summary": "...",
        "confidence": 0.85
    }}
"""


response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="sonar",
)

print("Raw Response:\n", response.choices[0].message.content)

Raw Response:
 {
  "stock": "PLTR",
  "sentiment": "Positive",
  "summary": "Despite a recent 5.4% daily price decline to $175.44, Palantir Technologies demonstrates strong fundamentals driven by its AI strategy, significant government contracts (including a $10 billion Pentagon deal), and robust commercial growth with 93% YoY revenue increase in the U.S. Its high valuation (PE around 373) and elevated volatility (beta ~2.6) present risks, yet expert forecasts and bullish analyst commentary suggest potential for continued upside, with AI-driven growth expected to sustain earnings growth and push the stock price potentially as high as $210 by end of 2025. Insider profit-taking from notable investors like Ken Griffin signals some short-term rebalancing but does not undermine AI sector confidence. Overall, the outlook remains positive amid valuation caution.",
  "confidence": 0.82
}


## Adding Memory to the planner operation as an example

### 1. add memories

In [19]:
from src.memory import MemoryWorker

# adding memories as data example
memory_worker = MemoryWorker()
memory_worker.execute('add', 'AAPL Q2 earnings beat expectations', ['AAPL', 'earnings'])
memory_worker.execute('add', 'AAPL announced a new AI partnership with NASA', ['AAPL', 'AI'])
memory_worker.execute('add', 'TSLA stock is volatile due to production concerns', ['TSLA', 'production'])

print("3 memories added.")

3 memories added.


### 2. ingest data

In [20]:
from src.memory import MemoryWorker


### Search Memories by tag

ticker = "AAPL"  # Example stock ticker
stock_info = FinancialDataIngestion().execute(ticker)
news_articles = NewsDataIngestion().execute(ticker, 5)
memories = memory_worker.execute('search', 'AAPL')


### Prompting for analysis

It demonstrates how the memory impacts the results. 
You can see that the added memories will appear on the summary

In [21]:

goal = f"Analyze {ticker} stock for a potential investment."

prompt = f"""
    As an autonomous investment research agent, create short sentiment analysis
      about the stock based on the context and recent news.
      Describe the sentiment as Positive, Negative, or Neutral.
      Give a summary of your analysis, and a confidence score between 0 and 1.
    Goal: {goal}

    Context: {stock_info}

    Recent News: {news_articles}

    Relevant Memories: {memories}

    Answer in JSON format:
    {{
        "stock": "...",
        "sentiment": "Positive",
        "summary": "...",
        "confidence": 0.85
    }}
"""


response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="sonar",
)

print("Raw Response:\n", response.choices[0].message.content)

Raw Response:
 {
  "stock": "AAPL",
  "sentiment": "Neutral",
  "summary": "Apple's stock (AAPL) is currently trading at $245.27, down about 3.45% on high volume relative to its recent average, signaling near-term selling pressure. The stock's recent price action shows a pullback from a 52-week high of $260.1, with notable technical support around $253.5. Despite the recent dip, Apple maintains strong fundamentals, a substantial market cap of over $3.6 trillion, a P/E ratio near 29.5, and continuing innovations including AI partnerships (e.g., with NASA) and technology acquisitions. Analyst sentiment is mixed but leaning neutral, with UBS keeping the price target at $220 and highlighting flattening iPhone demand, while technical analysis confirms a broader uptrend supported by ascending channel patterns and strong momentum indicators. Market news reflects some caution due to earnings and market conditions but also underscores ongoing strategic technology investments and stable financia

### Ingestion worker

In [22]:
from src.ingestion import Ingestion

ingestion_worker = Ingestion()
ingestion_worker.execute('AAPL')

{'symbol': 'AAPL',
 'timestamp': '2025-10-13T10:03:50.734357',
 'financial_data': {'current_price': 245.27000427246094,
  'price_change': -8.769995727539055,
  'price_change_pct': -3.452210568232977,
  'volume': 61782400,
  'avg_volume_30d': 54889995.45454545,
  'volatility_30d': 0.2566234162744101,
  'market_cap': 3639902470144,
  'pe_ratio': 29.515041,
  'dividend_yield': 0.42,
  '52_week_high': 260.1,
  '52_week_low': 169.21,
  'beta': 1.094,
  'sector': 'Technology',
  'industry': 'Consumer Electronics',
  'company_summary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App 